## 목표 : 빅마마 게임별 콜렉션별 특정 지표를 하이브 테이블로 제공하는 것

### 현황
* 모든 모바일 게임의 WAU (Weekly Active User) 지표를 서비스개발팀에서 일별 지표로 사용
* 기존 방식은 일별 BigMama 작업은 Spark Parquet 결과를 Spark Mongo Export 통해서 적재
  - Mongo Hadoop Connector 통해서 Hive 테이블 생성 활용하는 상황
* Mongo Hadoop Connector 가 MongoDB 버전이 올라가면서 사용할 수 없게 되는 상황
  - 모든 게임에 대해 WAU 작업만 매일 distcp + hive 작업을 생성해 주었고, 운영 중
  - MongoDB 작업은 수집기술팀에서, Hive 테이블 생성은 서비스개발팀에서 빅브라더를 통해 운영 중

<br>

### 이슈
* 미처 MAU 가 나오지 않는다는 사실이 최근에 리포팅, 기존에는 사용했던 지표
* 모든 게임에 대한 MAU (Monthly Active User) 까지 제공이 필요한 상황
* 단, 데이터서비스개발팀이 mongodb 에 직접 접근할 인터페이스가 없으며
* 외부 조직이 mongodb 에 직접 접근하는 것도 장애 혹은 보안적인 면에서 바람직하지도 않음

### 환경
* Hadoop 클러스터는 G2 전환 중이며, 별도로 운영 되고 있는 NCMC 하둡 클러스터입니다
* MongoDB 클러스터는 용량문제가 있어, 게임 별로 ncmc2, ncmc3 로 분리되어 운영되고 있습니다


#### 빅마마 게임별 콜렉션별 특정 지표를 하이브 테이블로 제공하는 것

| 단계 | 상세 | 설명 |
| --- | --- | --- |
| AS-IS | (wau, mau(+), ?) * games -> distcp -> (hive * games) | 실제 정보는 몽고디비에 존재, 빅브라더 작업생성 관리부담 증가 |
| TO-BE | mongo -> python -> pandas.dataframe -> pyarrow -> .parquet | 파라메터로 게임 목록을 받고 모든 게임 지표를 하나의 하이브 테이블로 제공 |



### 모든 게임 지표가 종료된 이후 시점에 모든 게임을 한 번에 
* 게임 별로 파라메터를 전달 받고, 모든 게임 지표를 동시에 수집
* 모든 지표를 하나의 테이블로 제공하되, 게임 및 지표 코드 컬럼을 추가

#### PR 과정에서 제안된 문제점 
> 빅마마 지표 장애 발생 시에 지표 지연 및 장애 발생 가능성 제기

- 게임 중에 하나라도 수집 실패하는 경우 성공한 게임은 지표가 지연되지 않도록 제공
- 해당 작업은 하나만 있지만, 상태를 실패로 남겨서 운영자가 인지할 수 있도록 할 것
- 빅마마 장애 복구 이후에 해당 작업을 복구하더라도 기존 지표에 영향이 없도록 덮어쓰도록 처리


In [28]:
columns = ["game_cd", "dt", "type", "value"]
df = pd.DataFrame([
    {"game_cd":"lw", "dt": "20220523", "type": "wau", "au": 1},
    {"game_cd":"lw", "dt": "20220501", "type": "mau", "au": 2},
])
df.columns = columns
df
list(df["dt"])

['20220523', '20220501']

In [14]:
df.dtypes

dt      object
type    object
xau      int64
dtype: object

In [20]:
df.query("value > 1")

,game_cd,dt,type,value
1,lw,20220501,mau,2


In [17]:
df.query("type.str.contains('au')")

,dt,type,xau
0,20220523,wau,1
1,20220501,mau,2



#### 참고
* main - https://git.ncsoft.net/projects/BIGMAMA/repos/bigmama-util-python/browse/scc_mobile_index_daily.py
* biz - https://git.ncsoft.net/projects/BIGMAMA/repos/bigmama-util-python/browse/core/MobileActiveUserDaily.py
* parquet - https://git.ncsoft.net/projects/BIGMAMA/repos/bigmama-util-python/browse/util/ParquetWriter.py
* https://arrow.apache.org/docs/python/filesystems.html#hadoop-distributed-file-system-hdfs
